In [24]:
from datasets import load_dataset, load_dataset_builder
import random as random

In [ ]:
ds = load_dataset("microsoft/ms_marco", 'v2.1', split="train")


In [26]:
random.seed(42)
size = len(ds)
print(f"Total size of dataset: {size}")
sample_size = 20000
sample_indices = random.sample(range(size), sample_size)
ds_sampled = ds.select(sample_indices)
print(f"Sampled dataset size: {len(ds_sampled)}")

Total size of dataset: 808731
Sampled dataset size: 20000


In [ ]:
print(ds_sampled.column_names)
print("\n")
for i in range(10):
    print(ds_sampled[i]['query'])
    print(ds_sampled[i]['answers'])
    print(ds_sampled[i]['wellFormedAnswers'])
    print('\n')

In [ ]:
#merge answers with wellFormedAnswers, and drop other columns
def simplify(example):
    if example['wellFormedAnswers'] and len(example['wellFormedAnswers']) > 0:
        answer = example['wellFormedAnswers'][0]
    else:
        answer = example['answers'][0]
    return {
        'query': example['query'],
        'answer': answer
    }
ds_simplified = ds_sampled.map(simplify, remove_columns=ds_sampled.column_names)
print(ds_simplified.column_names)
for i in range(10):
    print(ds_simplified[i])
    print('\n')


In [37]:
def has_multiple_sentences(example):
    period_count = example['answer'].count('.')
    return period_count > 1

ds_filtered = ds_simplified.filter(has_multiple_sentences)
print(f"Dataset size before filtering: {len(ds_simplified)}")
print(f"Dataset size after filtering: {len(ds_filtered)}")

for i in range(10):
    print(ds_filtered[i])
    print('\n')

Dataset size before filtering: 20000
Dataset size after filtering: 1544
{'query': 'cause of brief psychotic disorder', 'answer': 'Brief psychotic disorder is triggered by extreme stress, such as a traumatic accident or loss of a loved one. It is followed by a return to the previous level of function.'}


{'query': 'what are springs used for', 'answer': 'Springs are used to absorb energy or store it for long periods of time. You can find springs in everything from automatic doors to ballpoint pens.'}


{'query': 'what city is glacier national park in', 'answer': 'Glacier National Park is a national park located in the U.S. state of Montana, on the Canada–United States border with the Canadian provinces of Alberta and British Columbia'}


{'query': 'what are sea turtles classified as', 'answer': "Sea turtles sometimes called marine turtles, are reptiles of the order Testudines. The seven extant species of sea turtles are the green, loggerhead, Kemp's ridley, olive ridley, hawksbill, flat